In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import GridSearchCV

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
%matplotlib inline
from scipy import stats

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet

In [ ]:
model_vars_data = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp August\Machine Learning Project\data\model_vars_data_n.csv',index_col=0)

test_vars_data = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp August\Machine Learning Project\data\test_vars_data_n.csv',index_col=0)

price_train = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp August\Machine Learning Project\data\ny_train.csv',index_col=0)

price_test = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp August\Machine Learning Project\data\ny_test.csv',index_col=0)

In [ ]:
# list(model_vars_data.columns) 
model_vars_data = model_vars_data.drop(['MoSold','MSSubClass','LotArea'], axis = 1)
test_vars_data = test_vars_data.drop(['MoSold','MSSubClass','LotArea'], axis = 1) 
#there is a transformed version of LotArea


In [ ]:
pd.set_option('display.max_columns', None)
model_vars_data.head()


In [ ]:
pd.set_option('display.max_columns', None)
price_train.head()

In [ ]:
combine = pd.concat([price_train, model_vars_data], axis=1)
combine['log_price'] = np.log(combine.SalePrice)


In [ ]:
pd.set_option('display.max_columns', None)
model_vars_data.describe()

In [ ]:
pd.set_option('display.max_columns', None)
test_vars_data.describe()

In [ ]:
print(model_vars_data.shape)
print(price_train.shape)

In [ ]:
print(test_vars_data.shape)
print(price_test.shape)

In [ ]:
# Fucntion returns dictionary

def regx3(
model=  None,
x_train = None,
x_test = None,
y_train = None,
y_test =  None,
params = None,
cv_ = 5,
linear_reg = True,
tree_based = False):
######################################
#########################################

    features = list(x_train.columns)

    ss = StandardScaler()
    x_train = pd.DataFrame(ss.fit_transform(x_train), columns = features)
    x_test = pd.DataFrame(ss.fit_transform(x_test), columns = features)

    #     print('train ',x_train.head())
    #     print(' ')
    #     print('test ', x_test.head())


    gs = GridSearchCV(model, params, cv=cv_, return_train_score=True, refit=True)
    gs.fit(x_train,y_train)
    gs.best_params_

    reg_dict = {}
    model= gs.best_estimator_
    reg_dict['mod'] = model
    ###################################    

    print('best params: ',gs.best_params_)
    reg_dict['best params'] = gs.best_params_

    print('score: ',gs.score(x_train,y_train))
    reg_dict['score'] = gs.score(x_train,y_train)

    print('  ')
    print('test score: ',gs.score(x_test,y_test))
    reg_dict['test score'] = gs.score(x_test,y_test)

    if linear_reg:
        adj_r2 = 1-(1-gs.score(x_train,y_train))*(x_train.shape[0] - 1) / (x_train.shape[0] - x_train.shape[1] - 1)
        print('adj_r2: ', adj_r2)
        reg_dict['adj_r2'] = adj_r2

        adj_r2_test = 1-(1-gs.score(x_test,y_test))*(x_test.shape[0] - 1) / (x_test.shape[0] - x_test.shape[1] - 1)
        print('  ')
        print('adj_r2_test: ',adj_r2_test)
        reg_dict['adj_r2_test'] = adj_r2_test

        print(' ')

    train_pred = gs.best_estimator_.predict(x_train)
    print('train RMSE: ' + str(mean_squared_error(train_pred,y_train)**0.5))
    print('  ')
    reg_dict['train RMSE'] = mean_squared_error(train_pred,y_train)**0.5

    test_pred = gs.best_estimator_.predict(x_test)
    print('test RMSE: ' + str(mean_squared_error(test_pred,y_test)**0.5))
    ###
    reg_dict['test RMSE'] = mean_squared_error(test_pred,y_test)**0.5

    if linear_reg:

        coefs = pd.Series(gs.best_estimator_.coef_, name = 'coef' )

        varnames = pd.Series(features, name = 'features')

        pd.set_option('display.max_rows', None) # or 1000.

        abs_coefs = pd.Series(np.abs(coefs),name = 'abs_val')

        coefs_ = pd.concat([varnames,coefs, abs_coefs], axis=1)

        #return:
        model_importances = coefs_.sort_values(by=['abs_val'], ascending=False)
        model_importances.index=(range(model_importances.shape[0]))
        model_importances
        reg_dict['model_importances'] = model_importances

        print(model_importances)
        print(' ')

    elif tree_based:
        pd.set_option('display.max_rows', None) # or 1000.

        #return
        varnames = pd.Series(features, name = 'features')

        importances_ = pd.Series(model.feature_importances_, name = 'importances')

        df = pd.concat([varnames,importances_], axis=1)

        model_importances = df.sort_values(by=['importances'], ascending=False)

        model_importances.index=(range(model_importances.shape[0]))

        reg_dict['model_importances'] = model_importances



    #         model_importances = pd.Series(model.feature_importances_, index = features).sort_values(ascending=False)

        print(model_importances)


    if linear_reg:
        residuals = y_train - train_pred
        print('skewness: ',stats.skew(residuals))
        print(' ')
        sns.set_theme(style='darkgrid')
        sns.histplot(residuals, bins=20);

    print('model_importances.shape: ',model_importances.shape)
    
    return reg_dict


In [ ]:
def to_keep(mod, amt):
    mod = mod['model_importances']
    keeps1 = mod.loc[mod['abs_val'] > amt,['features']]
    list_keeps1 = list(keeps1.features)
    return list_keeps1


In [ ]:
lasso1 = regx3(model= Lasso(max_iter=20000),
x_train = model_vars_data,
x_test = test_vars_data,
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45.50,55,100],'tol':[0.0001]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
lasso1 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data,
x_test = test_vars_data,
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
lasso1a = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data,
x_test = test_vars_data,
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.03]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(lasso1, 0.000025)

In [ ]:
lasso2 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps_from_lasso2 = to_keep(lasso2, 0.000015)
keeps_from_lasso2

In [ ]:
lasso3 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
# 'Neighborhood_SawyerW',
dummy_drops = ['SaleCondition_Partial',
 'Exterior1st_BrkFace',
 'SaleCondition_Family',
 'MSSubClass_160',
 'Condition1_PosN',
 'LandContour_HLS',
 'MSZoning_RH',
 'Condition1_PosA',
 'Exterior2nd_CBlock',
 'RoofStyle_Mansard',
 'Exterior1st_PreCast',
 'MSSubClass_85',
 'RoofMatl_WdShngl',
 'Condition2_Feedr',
 'Foundation_Wood',
 'Exterior1st_CBlock',
 'Condition2_PosA',
 'BldgType_Twnhs',
 'MSSubClass_150',
 'Condition1_RRAn',
 'Heating_OthW',
 'HouseStyle_1.5Unf',
 'MSSubClass_90',
 'Heating_Grav',
 'MSSubClass_190',
 'Foundation_Stone',
 
 'Exterior2nd_Brk Cmn',
 'Heating_Wall',
 'LotConfig_FR3',
 'RoofMatl_Membran',
 'LotConfig_FR2',
 'Exterior2nd_AsphShn',
 'Condition1_RRNn',
 'RoofMatl_Tar&Grv',
 'Exterior2nd_PreCast',
 'HouseStyle_2.5Unf',
 'MSSubClass_75',
 'GarageType_Basment',
 'has_pool',
 'Exterior1st_WdShing',
 'MSSubClass_180',
 'Condition2_RRNn',
 'MSSubClass_40',
 'Fence_MnWw',
 'MoSold_11',
 'Exterior2nd_BrkFace',
 'Exterior2nd_Stone',
 'RoofStyle_Gambrel',
 'LandContour_Low',
 'Condition2_RRAn',
 'Electrical_FuseF',
 'SaleCondition_AdjLand',
 'SaleCondition_Alloca',
 'RoofStyle_Shed',
 'MSSubClass_30']

In [ ]:
lasso2_features = to_keep(lasso2, 0.00)
lasso2_features 

keep1 = set(lasso2_features) - set(dummy_drops)
lasso4_keeps = list(keep1)
# keeps

In [ ]:
lasso4 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
#drop features with corr < 0.15
lasso4_features = to_keep(lasso4, 0.00)
drop_corr_lt_015 = set(['Functional',
'ScreenPorch',
'KitchenAbvGr',
'OverallCond',
'3SsnPorch'
'BsmtHalfBath'
'MoSold'
'BsmtFinSF2'])

keeps_set =  set(lasso4_features) - drop_corr_lt_015
keeps_set
keeps = list(keeps_set)

In [ ]:
lasso5 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep = set(lasso4_keeps) - set(['MoSold'])
keeps_for_6 = list(keep)


In [ ]:
lasso6 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(lasso6, 0.002)

In [ ]:
lasso7 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(lasso7, 0.004)

In [ ]:
lasso8 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(lasso8, 0.006)


In [ ]:
lasso9 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(lasso9, 0.008)
lasso9_keeps = to_keep(lasso9,0)
# lasso9_keeps

In [ ]:
lasso10 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps_from_9 = to_keep(lasso9, 0.006)
keeps = keeps_from_9


In [ ]:
lasso11 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps_from_11 = to_keep(lasso11, 0.008)
keeps = keeps_from_11

lasso_11_feat = to_keep(lasso11, 0.00)
lasso_11_feat

In [ ]:
lasso12 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
lasso_11_feat = to_keep(lasso11, 0.00)
keeps = to_keep(lasso11,0.007)

In [ ]:
lasso13 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep_set = set(to_keep(lasso11,0)) - set(['log_first_FlrSF','BsmtFinSF1', 'BsmtUnfSF' ])
keeps = list(keep_set)

In [ ]:
lasso14 = regx3(model= Lasso(max_iter=40000), #max_iter=10000000000000
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,100],'tol':[0.02]},
cv_ = 5,
linear_reg = True,
tree_based = False)